<a href="https://colab.research.google.com/github/Surya152002/CFSOCR/blob/main/extract_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytesseract
!pip install pdf2image
!pip install pillow
!pip install python-docx


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=738aec0c6f31f1f3c140a1bee566df4cbbefd6eb4c8c096c4d1b4806b1414007
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [2]:
!apt-get install poppler-utils


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 28 not upgraded.
Need to get 174 kB of archives.
After this operation, 754 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 174 kB in 0s (608 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 122532 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.86.1-0ubuntu1.1_amd64.deb ...
Unpacking poppler-utils (0.86.1-0ubuntu1.1) ...
Setting up poppler-utils (0.86.1-0ubuntu1.1) ...
Processing triggers for man-db (2.9.1-1) ...


In [3]:
!apt-get install tesseract-ocr


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 28 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 0s (13.1 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 122562 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-eng_1%3a

In [5]:
!pip install img2table

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 13.3 MB/s eta 0:00:00


In [10]:
from img2table.document import PDF
from img2table.ocr import TesseractOCR

# Instantiation of the pdf
pdf = PDF(src="/content/diftab.pdf")

# Instantiation of the OCR, Tesseract, which requires prior installation
ocr = TesseractOCR(lang="eng")

# Table identification and extraction
pdf_tables = pdf.extract_tables(ocr=ocr)

# We can also create an excel file with the tables
pdf.to_xlsx('tables2.xlsx',
            ocr=ocr)

In [ ]:
import re
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
from docx import Document

# Step 1: Convert the PDF to images
pdf_path = '/content/table.pdf'
images = convert_from_path(pdf_path)

# Step 2: Process the images and extract text using OCR
extracted_text = []
for image in images:
    extracted_text.append(pytesseract.image_to_string(image))

# Step 3: Identify the table using regex pattern
table_pattern = r"((?:[^\n]*\n){3,}(?:[^\n]*\n){2,}[^\n]*)"
tables = re.findall(table_pattern, '\n'.join(extracted_text), re.MULTILINE)

# Step 4: Create a Word document for each table found
if tables:
    for i, table_text in enumerate(tables):
        # Remove invalid characters
        clean_table_text = ''.join(c for c in table_text if c.isprintable())
        
        document = Document()
        table = document.add_table(rows=1, cols=1)
        table.cell(0, 0).text = clean_table_text
        document.save(f'/content/table_{i+1}.docx')
    print(f"{len(tables)} table(s) extracted and saved as Word documents.")
else:
    print("No tables found in the PDF.")


1 table(s) extracted and saved as Word documents.


In [ ]:
import re
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment, Border, Side

# Step 1: Convert the PDF to images
pdf_path = '/content/table.pdf'
images = convert_from_path(pdf_path)

# Step 2: Process the images and extract text using OCR
extracted_text = []
for image in images:
    extracted_text.append(pytesseract.image_to_string(image))

# Step 3: Identify the table using regex pattern
table_pattern = r"((?:[^\n]*\n){3,}(?:[^\n]*\n){2,}[^\n]*)"
tables = re.findall(table_pattern, '\n'.join(extracted_text), re.MULTILINE)

# Step 4: Create an Excel workbook and save the tables
if tables:
    workbook = openpyxl.Workbook()
    for i, table_text in enumerate(tables):
        # Remove illegal characters
        clean_table_text = ''.join(c for c in table_text if c.isprintable() and c != '\x0c')
        
        worksheet = workbook.create_sheet(title=f'Table {i+1}')
        rows = clean_table_text.split('\n')
        
        for row_idx, row in enumerate(rows):
            columns = row.split('\t')
            
            for col_idx, col in enumerate(columns):
                clean_col = ''.join(c for c in col if c.isprintable())
                worksheet.cell(row=row_idx+1, column=col_idx+1, value=clean_col)
                
                cell = worksheet.cell(row=row_idx+1, column=col_idx+1)
                cell.border = Border(
                    left=Side(style='thin'),
                    right=Side(style='thin'),
                    top=Side(style='thin'),
                    bottom=Side(style='thin')
                )
                cell.alignment = Alignment(wrap_text=True)
                worksheet.column_dimensions[get_column_letter(col_idx+1)].width = 15
    
    # Remove the default sheet created by openpyxl
    workbook.remove(workbook['Sheet'])
    
    workbook.save('/content/tables.xlsx')
    print(f"{len(tables)} table(s) extracted and saved as tables.xlsx")
else:
    print("No tables found in the PDF.")


1 table(s) extracted and saved as tables.xlsx


In [ ]:
import pdf2image
import pytesseract
import cv2
import numpy as np

def extract_tables_from_pdf(pdf_path):
    images = pdf2image.convert_from_path(pdf_path)

    tables = []
    for i, image in enumerate(images):
        img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        extracted_text = pytesseract.image_to_string(img_cv)
        lines = extracted_text.split('\n')

        table_lines = []
        prev_y = None
        for line in lines:
            if line.strip():
                try:
                    y = int(line.split(' ')[0])
                    if prev_y is None:
                        prev_y = y
                    elif abs(y - prev_y) > 50:  # Adjust the threshold as per your needs
                        table_lines.append(line)
                    prev_y = y
                except ValueError:
                    continue

        if table_lines:
            tables.append(table_lines)

    return tables

def print_tables(tables):
    for i, table in enumerate(tables):
        print(f"Table {i+1}:")
        for line in table:
            print(line)

# Usage example
pdf_file = '/content/table.pdf'
tables = extract_tables_from_pdf(pdf_file)
print_tables(tables)


In [ ]:
import pandas as pd
from docx import Document

# Read data from Excel file
excel_file = '/content/tables2.xlsx'
df = pd.read_excel(excel_file)

# Create a new Word document
doc = Document()

# Create a table in the Word document
table = doc.add_table(rows=df.shape[0]+1, cols=df.shape[1])
table.style = 'Table Grid'

# Set the column headers
for i, column_name in enumerate(df.columns):
    table.cell(0, i).text = column_name

# Fill in the data rows
for i, row in df.iterrows():
    for j, value in enumerate(row):
        table.cell(i+1, j).text = str(value)

# Save the Word document
word_file = '/content/file9.docx'
doc.save(word_file)

print('Data extracted from Excel and pasted into Word in table format.')


Data extracted from Excel and pasted into Word in table format.


In [ ]:
import openpyxl
from docx import Document
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.table import WD_CELL_VERTICAL_ALIGNMENT

def excel_to_word(excel_file, word_file):
    # Load the Excel file
    wb = openpyxl.load_workbook(excel_file)
    sheet = wb.active

    # Get the data from the Excel sheet
    data = []
    for row in sheet.iter_rows(values_only=True):
        data.append(row)

    # Create a Word document
    doc = Document()

    # Populate the Word document with the Excel data
    for row in data:
        table = doc.add_table(rows=1, cols=len(row))
        table.autofit = False
        
        # Apply table style
        table.style = 'Table Grid'  # Replace with the desired table style
        
        # Set table alignment
        table.alignment = WD_TABLE_ALIGNMENT.CENTER
        
        # Set vertical alignment for cells
        for cell in table.columns[0].cells:
            cell.vertical_alignment = WD_CELL_VERTICAL_ALIGNMENT.CENTER
        
        for i, cell_value in enumerate(row):
            cell = table.cell(0, i)
            cell.text = str(cell_value)

    # Save the Word document
    doc.save(word_file)

# Usage example
excel_file = '/content/tables2.xlsx'
word_file = '/content/file.docx'
excel_to_word(excel_file, word_file)


In [ ]:
import pandas as pd
from docx import Document

# Read data from Excel file
excel_file = '/content/tables2.xlsx'
df = pd.read_excel(excel_file)

# Create a new Word document
doc = Document()

# Create a table in the Word document
table = doc.add_table(rows=df.shape[0]+1, cols=df.shape[1])
table.style = 'Table Grid'

# Set the column headers
for i, column_name in enumerate(df.columns):
    table.cell(0, i).text = column_name

# Fill in the data rows
for i, row in df.iterrows():
    for j, value in enumerate(row):
        table.cell(i+1, j).text = str(value)

# Save the Word document
word_file = '/content/file8.docx'
doc.save(word_file)

print('Data extracted from Excel and pasted into Word in table format.')


Data extracted from Excel and pasted into Word in table format.
